In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import xml.etree.ElementTree as ET
from collections import defaultdict
import requests

# Dette publique

In [288]:
# Load the data from the Excel file
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
# Importer les données depuis l'URL
dette_publique_url = 'https://sebastien-hein.emi.u-bordeaux.fr/OI-sbzrthstrm/DATA/dette_pub.xlsx'
code_url = 'https://sebastien-hein.emi.u-bordeaux.fr/OI-sbzrthstrm/DATA/code.tsv'

# Charger les fichiers directement depuis l'URL
dette_publique = pd.read_excel(dette_publique_url, sheet_name='Feuille 1')
code = pd.read_csv(code_url, sep='\t')

# Define column names for code and label
col_code = 'CODE'
col_label = 'Label - French'

def find_value(x):
    """Find the label value based on the code."""
    matched = not code[code[col_code] == x].empty
    if matched:
        return code[code[col_code] == x].loc[:, col_label].iloc[0]
    elif x == 'TIME':
        return 'Country'
    return None

# Modify the index of the dataframe
dette_publique.index = dette_publique.iloc[:, 0].apply(find_value)
dette_publique.index.name = None

# Set the column names based on the 'Country' row
dette_publique.columns = dette_publique.loc['Country']

# Filter the dataframe to include only rows from 'Belgique' to 'Suède' onwards and exclude the 'TIME' column
# Indeed, only the countries interest us,
# and the datas are missing for Islande, Norvège, Suisse and United Kingdom.
dette_publique = dette_publique.loc['Belgique':'Suède', dette_publique.columns != 'TIME']

# Filter the dataframe to include only columns from the year 2002 onwards
dette_publique = dette_publique.loc[:,2002:] # Problème non résolu: Si l'on prend une date inférieure à 2002, 
                                             #                      l'interpolation ne fonctionne nul part.

def to_date(x):
    """Convert a value to datetime."""
    return pd.to_datetime(x, format='%Y')

# Vectorize the to_date function
vect_to_date = np.vectorize(to_date)

# Convert the columns to datetime
dette_publique.columns = vect_to_date(dette_publique.columns.values)

monthly_dates = pd.date_range(start=dette_publique.columns.values[0], end=dette_publique.columns.values[-1], freq='MS')

# Add columns for each month from 2013 to 2023
dette_publique = dette_publique.reindex(columns=dette_publique.columns.union(monthly_dates))

def fill_val(x):
    """Fill missing values by resampling and interpolating."""
    return x.resample('MS').interpolate(method='quadratic')

# Apply the fill_val function to each row
dette_publique = dette_publique.apply(func=fill_val, axis=1).T

dette_publique.isna().sum().sum() # Number of missing values (0)

np.int64(0)

# IPCH

In [334]:
# Define URLs for the data sources
ipch_url = 'https://sebastien-hein.emi.u-bordeaux.fr/OI-sbzrthstrm/DATA/ipch.tsv'
code_cp_url = 'https://sebastien-hein.emi.u-bordeaux.fr/OI-sbzrthstrm/DATA/code_cp.tsv'

# Load the data directly from the URLs
ipch = pd.read_csv(ipch_url, sep='\t')  # Load the ipch data using tab as a separator
code_cp = pd.read_csv(code_cp_url, sep='\t')  # Load the code_cp data using tab as a separator


# Set index
def get_CP(x): return x[8:12]  # Extract CP code
def get_id(x): return x[13:]  # Extract id
vect_get_cp = np.vectorize(get_CP)
vect_get_id = np.vectorize(get_id)
ipch['CP'] = vect_get_cp(ipch.iloc[:, 0])  # Apply CP extraction
ipch['id'] = vect_get_id(ipch.iloc[:, 0])  # Apply id extraction

ipch = ipch[ipch['CP'] == 'CP00']
ipch.drop(columns = 'CP', inplace = True)

ipch.drop(columns='freq,unit,coicop,geo\\TIME_PERIOD', inplace=True)  # Drop unnecessary columns
ipch['country'] = ipch.loc[:, 'id'].apply(find_value)  # Find country names
ipch.set_index(['country'], inplace=True)  # Set index
ipch.drop(columns='id', inplace=True)  # Drop id column

# Convert the columns to datetime
def to_date_M(x):
    """Convert a value to datetime."""
    try:
        return pd.to_datetime(x[:-1], format='%Y-%m')
    except:
        print('fail')
        return x

vect_to_date_M = np.vectorize(to_date_M)
ipch.columns = vect_to_date_M(ipch.columns.values)  # Apply datetime conversion

# Filter data
ipch = ipch[~ipch.index.str.startswith(('Union', 'Zone', 'Espace'))]  # Exclude certain countries

# Clean and convert to numeric
ipch = ipch.map(lambda x: pd.to_numeric(
    str(x).replace(' ', '').replace('d', ''), errors='coerce'))
ipch = ipch.T

# Missing values
# Initialiser le dictionnaire pour stocker les plages de dates manquantes
missing_ranges = defaultdict(list)

# Identifier les valeurs manquantes
missing_values = ipch.isna()

# Parcourir chaque pays (colonne) pour trouver les plages de dates manquantes
for country in ipch.columns:
    country_missing = missing_values[country]
    if not country_missing.empty:
        # Trouver les plages de dates manquantes
        missing_dates = country_missing[country_missing].index
        start_date = None
        for date in missing_dates:
            if start_date is None:
                start_date = date
            if (date + pd.DateOffset(months=1)) not in missing_dates:
                end_date = date
                missing_ranges[country].append((start_date.strftime('%Y-%m'), end_date.strftime('%Y-%m')))
                start_date = None

# Convertir le defaultdict en dict
missing_ranges = dict(missing_ranges)

for country, dates in missing_ranges.items():
    print(f'{country}: {dates}')

# Delete the country for which the missing data is on a bigger period than 4 years
ipch.drop(columns = 'Albanie, Kosovo*, Monténégro'.split(', '), inplace = True)

# Fill missing values using interpolation as the most consistent method for long gaps
ipch.interpolate(method='time', inplace=True, limit_direction='both')  # Interpolate linearly by date for smoother transitions

# Optionally fill remaining missing values (if interpolation failed for some edge cases) with column mean
ipch.fillna(ipch.mean(), inplace=True)

ipch.isna().sum().sum() # Number of missing values (0)

Albanie: [('2010-12', '2016-11')]
Monténégro: [('2010-12', '2015-11')]
United Kingdom: [('2020-12', '2024-11')]
Kosovo*: [('2010-12', '2016-11')]


np.int64(0)

# Chomage

In [290]:
def parse_chomage_xml(file_url):
    """Parse XML file and extract data into a DataFrame."""
    
    # Download the XML file content using requests
    response = requests.get(file_url)
    xml_content = response.text  # Get the content of the XML file as a string
    
    # Parse the XML content with ElementTree
    root = ET.fromstring(xml_content)  # Parse the XML string directly
    
    # Initialize lists to store data
    data = []
    columns = set()
    rows = set()

    # Extract data from <Series> and <Obs> tags
    for series in root.findall('.//Series'):
        geo = series.attrib.get('geo')  # Get "geo" attribute
        if geo:
            rows.add(geo)
            for obs in series.findall('Obs'):
                time_period = obs.attrib.get('TIME_PERIOD')  # Get "TIME_PERIOD" attribute
                obs_value = obs.attrib.get('OBS_VALUE')  # Get "OBS_VALUE" attribute
                if time_period and obs_value:
                    columns.add(time_period)
                    data.append((geo, time_period, obs_value))

    # Create DataFrame with appropriate indices
    df = pd.DataFrame(index=sorted(rows), columns=sorted(columns))

    # Fill DataFrame with extracted values
    for geo, time_period, obs_value in data:
        df.at[geo, time_period] = obs_value

    return df

# URL for the XML data
file_url = 'https://sebastien-hein.emi.u-bordeaux.fr/OI-sbzrthstrm/DATA/chomage.xml'

# Load the data
chomage = parse_chomage_xml(file_url)

# Format the data
chomage.columns = chomage.columns.map(lambda x: \
                                      pd.to_datetime(x, format='%Y-%m'))  # Convert columns to datetime
chomage.index = chomage.index.map(lambda x: \
                code.loc[code.loc[:, 'CODE'] == x, 'Label - French'].iloc[0])  # Map index to labels
chomage.drop('Zone euro - 20 pays (à partir de 2023)', inplace=True)  # Drop specific rows
chomage.drop('Union européenne - 27 pays (à partir de 2020)', inplace=True)  # Drop specific rows
chomage = chomage.apply(pd.to_numeric).T  # Convert data to numeric


# Missing values
missing_ranges = defaultdict(list) # Initialiser le dictionnaire pour stocker les plages de dates manquantes
missing_values = chomage.isna() # Identifier les valeurs manquantes

for country in chomage.columns: # Parcourir chaque pays (colonne) pour trouver les plages de dates manquantes
    country_missing = missing_values[country]
    if country_missing.any():
        # Trouver les plages de dates manquantes
        missing_dates = country_missing[country_missing].index
        start_date = None
        for date in missing_dates:
            if start_date is None:
                start_date = date
            if date + pd.DateOffset(months=1) not in missing_dates:
                end_date = date
                missing_ranges[country].append((start_date.strftime('%Y-%m'), 
                                                end_date.strftime('%Y-%m')))
                start_date = None

# Fill missing values using interpolation as the most consistent method for long gaps
chomage.interpolate(method='time', inplace=True, limit_direction='both')  # Interpolate linearly by date for smoother transitions

# Optionally fill remaining missing values (if interpolation failed for some edge cases) with column mean
chomage.fillna(chomage.mean(), inplace=True)

ipch.isna().sum().sum() # Number of missing values (0)

np.int64(0)

# Fusion des dataframe

In [291]:
def add_index(df, index_name):
    if isinstance(df.columns, pd.MultiIndex):
        # Ajouter le nouvel index au niveau supérieur du MultiIndex existant
        new_index = pd.MultiIndex.from_tuples([(index_name, *idx) \
                        for idx in df.columns], names=['Type'] + df.columns.names)
    else:
        # Créer un MultiIndex en juxtaposant le nouvel index et l'index existant
        new_index = pd.MultiIndex.from_tuples([(index_name, idx) 
                        for idx in df.columns], names=['Type', df.columns.names[0]])
    df.columns = new_index
    return df
dette_publique = add_index(dette_publique, 'Dette')
chomage = add_index(chomage, 'Chomage')
ipch = add_index(ipch, 'IPCH')

data = pd.concat((dette_publique, chomage, ipch), axis = 1, join = 'inner')
data = data.groupby('Type', axis = 1)


/var/folders/dh/8spvhwwd3gjgp2cl9zvhb_4m0000gn/T/ipykernel_32258/246048158.py:17: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data = data.groupby('Type', axis = 1)


In [294]:
data.get_group('IPCH')

Type          IPCH                                                    \
           Albanie Autriche Belgique Bulgarie Suisse Chypre Tchéquie   
2013-01-01     2.5      2.8      1.5      2.6   -0.1    2.0      2.0   
2013-02-01     2.5      2.6      1.5      2.2    0.1    1.8      1.7   
2013-03-01     2.5      2.4      1.4      1.6   -0.3    1.3      1.5   
2013-04-01     2.5      2.1      1.2      0.9   -0.4    0.1      1.7   
2013-05-01     2.5      2.4      1.2      1.0   -0.2    0.1      1.2   
...            ...      ...      ...      ...    ...    ...      ...   
2022-09-01     7.9     11.0     12.1     15.6    3.2    9.0     17.8   
2022-10-01     8.0     11.6     13.1     14.8    2.9    8.6     15.5   
2022-11-01     8.0     11.2     10.5     14.3    2.9    8.1     17.2   
2022-12-01     7.5     10.5     10.2     14.3    2.7    7.6     16.8   
2023-01-01     7.1     11.6      7.4     14.3    3.2    6.8     19.1   

Type                                   ...                                 \
           Allemagne Danemark Estonie  ... Portugal Roumanie Serbie Suède   
2013-01-01       1.9      0.9     3.7  ...      0.4      5.1   12.7   0.6   
2013-02-01       1.8      1.1     4.0  ...      0.2      4.9   12.4   0.5   
2013-03-01       1.9      0.7     3.8  ...      0.7      4.4   11.2   0.5   
2013-04-01       1.1      0.5     3.4  ...      0.4      4.4   11.5   0.0   
2013-05-01       1.5      0.7     3.6  ...      0.9      4.4    9.5   0.3   
...              ...      ...     ...  ...      ...      ...    ...   ...   
2022-09-01      10.9     11.1    24.1  ...      9.8     13.4   13.6  10.3   
2022-10-01      11.6     11.4    22.5  ...     10.6     13.5   14.6   9.8   
2022-11-01      11.3      9.7    21.4  ...     10.2     14.6   14.5  10.1   
2022-12-01       9.6      9.6    17.5  ...      9.8     14.1   14.6  10.8   
2023-01-01       9.2      8.4    18.6  ...      8.6     13.4   15.2   9.6   

Type                                                                     
           Slovénie Slovaquie Turquie United Kingdom États-Unis Kosovo*  
2013-01-01      2.8       2.5     7.4            2.7        1.5     1.3  
2013-02-01      2.9       2.3     7.2            2.8        2.0     1.3  
2013-03-01      2.2       1.9     7.5            2.8        1.4     1.3  
2013-04-01      1.6       1.7     6.0            2.4        0.8     1.3  
2013-05-01      1.6       1.8     6.3            2.7        1.2     1.3  
...             ...       ...     ...            ...        ...     ...  
2022-09-01     10.6      13.6    83.4            0.3        8.7    12.7  
2022-10-01     10.3      14.5    85.4            0.3        8.0    12.7  
2022-11-01     10.8      15.1    84.4            0.3        7.1    11.6  
2022-12-01     10.8      15.0    64.3            0.3        6.1    12.1  
2023-01-01      9.9      15.1    57.7            0.3        6.0    12.0  

[121 rows x 39 columns]

In [282]:
common_countries = set(ipch.index.get_level_values(1)) &\
                   set(dette_publique.index.get_level_values(1)) &\
                   set(chomage.index.get_level_values(1))
common_countries = pd.Index(common_countries)
common_dates = set(ipch.columns) &\
               set(dette_publique.columns) &\
               set(chomage.columns)
common_dates = pd.Index(common_dates)
common_countries

IndexError: Too many levels: Index has only 1 level, not 2

# Affichage

In [5]:
import ipywidgets as widgets
from ipywidgets import interact_manual
import datetime


# Widgets pour sélectionner les pays et les dates
countries_widget = widgets.Text(
    value='France, Allemagne, Italie',
    description='Countries:',
    placeholder='Enter countries separated by commas'
)

start_date_widget = widgets.DatePicker(
    value=pd.to_datetime('2015-1', format = '%Y-%m'),
    description='Start Date'
)

end_date_widget = widgets.DatePicker(
    value=pd.to_datetime('2020-3', format = '%Y-%m'),
    description='End Date'
)

# Fonction pour tracer la dette publique
@interact_manual(countries=countries_widget, start_date=start_date_widget, end_date=end_date_widget)
def plot_dette(countries, start_date, end_date):
    countries = countries.split(', ')
    dates = pd.date_range(start_date, end_date, freq='MS')
    
    plt.figure(figsize=(9, 4))
    for country in countries:
        if country in dette_publique.index:
            dette_publique.loc[country, dates].plot(label=country)
    plt.title(f'Dette publique ({dates[0].strftime("%m/%Y")} - {dates[-1].strftime("%m/%Y")})')
    plt.legend()
    plt.show()

# Fonction pour tracer l'inflation
@interact_manual(countries=countries_widget, start_date=start_date_widget, end_date=end_date_widget)
def plot_inflation(countries, start_date, end_date):
    countries = countries.split(', ')
    dates = pd.date_range(start_date, end_date, freq='MS')
    
    plt.figure(figsize=(9, 4))
    for country in countries:
        if country in ipch.get_group('CP00').index:
            ipch.get_group('CP00').loc[country, dates].iloc[0].plot(label=country)
    plt.title(f'Inflation ({dates[0].strftime("%m/%Y")} - {dates[-1].strftime("%m/%Y")})')
    plt.legend()
    plt.show()

# Fonction pour tracer le chômage
@interact_manual(countries=countries_widget, start_date=start_date_widget, end_date=end_date_widget)
def plot_chomage(countries, start_date, end_date):
    countries = countries.split(', ')
    dates = pd.date_range(start_date, end_date, freq='MS')
    
    plt.figure(figsize=(9, 4))
    for country in countries:
        if country in chomage.index:
            chomage.loc[country, dates].plot(label=country)
    plt.title(f'Chômage ({dates[0].strftime("%m/%Y")} - {dates[-1].strftime("%m/%Y")})')
    plt.legend()
    plt.show()

interactive(children=(Text(value='France, Allemagne, Italie', continuous_update=False, description='Countries:…

interactive(children=(Text(value='France, Allemagne, Italie', continuous_update=False, description='Countries:…

interactive(children=(Text(value='France, Allemagne, Italie', continuous_update=False, description='Countries:…

<Figure size 900x400 with 0 Axes>